In [8]:
import json
import boto3
from typing import Dict

SECRET_NAME = "ds4a-project-dev-db-connection-secret"


def get_secret(aws: boto3.Session, secret_name: str) -> Dict[str, str]:
    sm = aws.client("secretsmanager")
    response = sm.get_secret_value(SecretId=secret_name)
    secret = response["SecretString"]
    return json.loads(secret)


aws = boto3.Session()
params = get_secret(aws, SECRET_NAME)
params.keys()


dict_keys(['password', 'dbname', 'engine', 'port', 'dbInstanceIdentifier', 'host', 'username'])

In [9]:
import psycopg2


conn = psycopg2.connect(
    host=params["host"],
    database=params["dbname"],
    user=params["username"],
    password=params["password"],
)


In [6]:
cur = conn.cursor()
type(cur)


psycopg2.extensions.cursor

In [7]:
type(conn)


psycopg2.extensions.connection

In [10]:
cur = conn.cursor()
cur.execute("SELECT version()")
cur.fetchone()


('PostgreSQL 13.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-6), 64-bit',)

In [9]:
create_statement = """--sql
CREATE TABLE IF NOT EXISTS departments (
    code INTEGER NOT NULL,
    name VARCHAR(30) NOT NULL,
    PRIMARY KEY (code)
)
"""

cur.execute(create_statement)
conn.commit()


In [10]:
query = """--sql
SELECT
  column_name, data_type, is_nullable
FROM
  INFORMATION_SCHEMA.COLUMNS
WHERE
  TABLE_NAME = 'departments';
"""
cur.execute(query)
cur.fetchall()


[('code', 'integer', 'NO'), ('name', 'character varying', 'NO')]

In [11]:
cur.execute("""--sql
DROP TABLE departments
""")
conn.commit()


In [7]:
conn.rollback()

In [11]:
import pathlib
import pandas as pd


data_dir = pathlib.Path.home() / "ds4a" / "project" / "infrastructure" / "data"
dept_pop = pd.read_csv(
    data_dir / "department_population_2016_2022.csv",
    sep=";",
    usecols=["DP", "DPNOM", "AÑO", "Total Hombres", "Total Mujeres", "Total"],
)

departments = dept_pop[["DP", "DPNOM"]].drop_duplicates()


In [15]:
for x in departments.to_numpy():
    print(x)


[5 'Antioquia']
[8 'Atlántico']
[11 'Bogotá, D.C.']
[13 'Bolívar']
[15 'Boyacá']
[17 'Caldas']
[18 'Caquetá']
[19 'Cauca']
[20 'Cesar']
[23 'Córdoba']
[25 'Cundinamarca']
[27 'Chocó']
[41 'Huila']
[44 'La Guajira']
[47 'Magdalena']
[50 'Meta']
[52 'Nariño']
[54 'Norte de Santander']
[63 'Quindio']
[66 'Risaralda']
[68 'Santander']
[70 'Sucre']
[73 'Tolima']
[76 'Valle del Cauca']
[81 'Arauca']
[85 'Casanare']
[86 'Putumayo']
[88 'Archipiélago de San Andrés']
[91 'Amazonas']
[94 'Guainía']
[95 'Guaviare']
[97 'Vaupés']
[99 'Vichada']


In [16]:
import traceback
from psycopg2.extras import execute_batch

insert_statement = """--sql
INSERT INTO departments (code, name) VALUES (%s, %s)
"""

try:
    execute_batch(cur, insert_statement, departments.to_numpy())
    conn.commit()
except Exception as e:
    tb = traceback.format_exc()
    print(tb)
    conn.rollback()


In [17]:
query = """--sql
SELECT count(*) FROM departments
"""

cur.execute(query)
cur.fetchall()


[(33,)]

In [26]:
from psycopg2.extras import RealDictCursor

query = """--sql
SELECT * FROM departments
"""

cursor = conn.cursor(cursor_factory=RealDictCursor)
cursor.execute(query)
res = cursor.fetchall()
res


[RealDictRow([('code', 5), ('name', 'Antioquia')]),
 RealDictRow([('code', 8), ('name', 'Atlántico')]),
 RealDictRow([('code', 11), ('name', 'Bogotá, D.C.')]),
 RealDictRow([('code', 13), ('name', 'Bolívar')]),
 RealDictRow([('code', 15), ('name', 'Boyacá')]),
 RealDictRow([('code', 17), ('name', 'Caldas')]),
 RealDictRow([('code', 18), ('name', 'Caquetá')]),
 RealDictRow([('code', 19), ('name', 'Cauca')]),
 RealDictRow([('code', 20), ('name', 'Cesar')]),
 RealDictRow([('code', 23), ('name', 'Córdoba')]),
 RealDictRow([('code', 25), ('name', 'Cundinamarca')]),
 RealDictRow([('code', 27), ('name', 'Chocó')]),
 RealDictRow([('code', 41), ('name', 'Huila')]),
 RealDictRow([('code', 44), ('name', 'La Guajira')]),
 RealDictRow([('code', 47), ('name', 'Magdalena')]),
 RealDictRow([('code', 50), ('name', 'Meta')]),
 RealDictRow([('code', 52), ('name', 'Nariño')]),
 RealDictRow([('code', 54), ('name', 'Norte de Santander')]),
 RealDictRow([('code', 63), ('name', 'Quindio')]),
 RealDictRow([('c